# Rapport Projet BOMR

Authors

Valentin Suppa-Gallezot   
Mehdi El Bouari   
Thibault Golaz  
Louis Marchand  


## Introduction

## 1. Vision

## Most important function in Vision

| Function of Vision | Input | Output |
|:------|:------|:------|
|`detect_start(image, show=False, begin = True)` |receive a frame when the program starts, show allows to plot the image and begin allows to get only the start_coordinates|An image with a white rectangle on the two circles of the thymio and the start coordinates|
|`detect_target(image)`|receive a frame when the program starts|An image with a white rectangle on the target and the coordinates of the target|
|`detect_obstacle(image)`|receive an image with white rectangle on the start and the target|corner coordinates of each obstacle|

### Start and target Detection 

In order to obtain the starting point and the target point, we used two colored circles (1 blue and 1 green) of different sizes. These two circles are placed on the robot. At the beginning of the program, a picture of the scene is taken. Using the library opencv and more precisely the function template matching the center of the two circles are detected. Then by just computing the middle of these centers the center of the thymio (called starting point in the program is detected). Two circles are needed because in the following of the code it will allow to compute the orientation of the thymio  
The same method is used to get the target but a red rectangle is used instead.

In [ ]:
### code refer

Since the starting point and the goal are detected it is time to compute the coordinates of the obstacle on the map. In our project the obstacles are black 2 dimensions shapes. They are positionned randomly on the field. In order to detect them the image is converted in black and white. To avoid some risk of confusion with the shapes on the thymio and the one for the target, white rectangles are drawn on them. Then, after applying the threshold, the obstacle are detected (#plot image). By using an opencv function, the coordinates of the contours are found. Finaylly by following the steps of this link https://stackoverflow.com/questions/50984205/how-to-find-corners-points-of-a-shape-in-an-image-in-opencv every corner coordinates of each shape are stored in a array. It will be helpful to compute the visibility graph.  

## 2. Global Navigation 

## 3. Motion Control

## 4. Local avoidance

## 5. Filtering

## 6. Conclusion